In [5]:
import sys
print(sys.version)

3.10.14 (main, May  6 2024, 14:47:20) [Clang 14.0.6 ]


### streetview: a package that gathers panorama street images given latitude and longitude.
- [GitHub Repository](https://github.com/robolyst/streetview)
- *Installation:*
    ```bash
    pip install streetview
    ```
### `streetview.search_panoramas()`

- *Input:*
    1. ***latitude***
    2. ***longitude***

- *Output:*
    *A list of information tuples. The first element, `panos[0]`, returns crucial info:*
    1. ***pano_id:*** *a key for each image data in the source pool*
    2. ***lat:*** *latitude of the location*
    3. ***lon:*** *longitude of the location*
    4. ***heading:*** *The compass heading, or the direction the camera is facing, measured in degrees from true north.*
    5. ***pitch:*** *The angle at which the camera is tilted up or down.*
    6. ***roll:*** *The angle at which the camera is tilted sideways.*
    7. ***date:*** *The date when the panorama was captured.*
    8. ***Elevation:*** *The elevation of the camera when the panorama was taken. If this is None, it means the elevation information is not provided or not applicable.*


In [7]:
from streetview import search_panoramas
panos = search_panoramas(lat=43.156573, lon=-77.608849)
first = panos[0]
print(first)

pano_id='B5LQCXCjt-ZBgQDVD062Xg' lat=43.156551040182 lon=-77.60884812843123 heading=77.48731231689453 pitch=90.15473175048828 roll=1.819120526313782 date='2015-07' elevation=None


### `streetview.fetch_panorama()`

- **Input:**
  - ***pano_id:*** *the unique key you obtained using `search_panoramas()`*

- **Output:**
  - *an image entity, you could use `image.save(dir, format)`*


### Monitoring Space and Time Taken

To monitor the space and time taken for a request, use the `memory_profiler` and `time` modules.

- **Install `memory_profiler`:**

  ```bash
  pip install memory_profiler

In [10]:
from time import time
from memory_profiler import memory_usage
from streetview import get_panorama

def execute_and_profile():
    # Start timing
    start_time = time()
    
    # Function to measure memory usage
    def fetch_panorama():
        image = get_panorama(pano_id='B5LQCXCjt-ZBgQDVD062Xg')
        image.save('/Users/klz/Desktop/Cantay_Panorama/input_pano/example.jpg', "jpeg")

    mem_usage = memory_usage(fetch_panorama)
    end_time = time()
    time_taken = end_time - start_time
    
    return time_taken, mem_usage

time_taken, mem_usage = execute_and_profile()

print(f"Time taken: {time_taken:.2f} seconds")
print(f"Memory usage: {max(mem_usage) - min(mem_usage):.2f} MiB")


Time taken: 100.19 seconds
Memory usage: 511.01 MiB


### Converting Panorama Image to Perspective Image

Now, we need to convert a panorama image into a perspective image. This process is called **Equirectangular projection**: *Directly maps the longitude to the x-coordinate and latitude to the y-coordinate.*

Thanks to: https://github.com/fuenwang/Equirec2Perspec


In [9]:
import os
import sys
import cv2
import numpy as np
from time import time
from memory_profiler import memory_usage

def xyz2lonlat(xyz):
    atan2 = np.arctan2
    asin = np.arcsin

    norm = np.linalg.norm(xyz, axis=-1, keepdims=True)
    xyz_norm = xyz / norm
    x = xyz_norm[..., 0:1]
    y = xyz_norm[..., 1:2]
    z = xyz_norm[..., 2:]

    lon = atan2(x, z)
    lat = asin(y)
    lst = [lon, lat]

    out = np.concatenate(lst, axis=-1)
    return out

def lonlat2XY(lonlat, shape):
    X = (lonlat[..., 0:1] / (2 * np.pi) + 0.5) * (shape[1] - 1)
    Y = (lonlat[..., 1:] / (np.pi) + 0.5) * (shape[0] - 1)
    lst = [X, Y]
    out = np.concatenate(lst, axis=-1)

    return out 

class Equirectangular:
    def __init__(self, img_name):
        self._img = cv2.imread(img_name, cv2.IMREAD_COLOR)
        [self._height, self._width, _] = self._img.shape

    def GetPerspective(self, FOV, THETA, PHI, height, width):
        #
        # THETA is left/right angle, PHI is up/down angle, both in degree
        #

        f = 0.5 * width * 1 / np.tan(0.5 * FOV / 180.0 * np.pi)
        cx = (width - 1) / 2.0
        cy = (height - 1) / 2.0
        K = np.array([
                [f, 0, cx],
                [0, f, cy],
                [0, 0,  1],
            ], np.float32)
        K_inv = np.linalg.inv(K)
        
        x = np.arange(width)
        y = np.arange(height)
        x, y = np.meshgrid(x, y)
        z = np.ones_like(x)
        xyz = np.concatenate([x[..., None], y[..., None], z[..., None]], axis=-1)
        xyz = xyz @ K_inv.T

        y_axis = np.array([0.0, 1.0, 0.0], np.float32)
        x_axis = np.array([1.0, 0.0, 0.0], np.float32)
        R1, _ = cv2.Rodrigues(y_axis * np.radians(THETA))
        R2, _ = cv2.Rodrigues(np.dot(R1, x_axis) * np.radians(PHI))
        R = R2 @ R1
        xyz = xyz @ R.T
        lonlat = xyz2lonlat(xyz) 
        XY = lonlat2XY(lonlat, shape=self._img.shape).astype(np.float32)
        persp = cv2.remap(self._img, XY[..., 0], XY[..., 1], cv2.INTER_CUBIC, borderMode=cv2.BORDER_WRAP)

        return persp

def profile_conversion(img_path, save_path, FOV, THETA, PHI, height, width):
    def fetch_panorama():
        equi = Equirectangular(img_path)
        image = equi.GetPerspective(FOV, THETA, PHI, height, width)
        cv2.imwrite(save_path, image)
    
    start_time = time()
    mem_usage = memory_usage(fetch_panorama)
    end_time = time()
    
    time_taken = end_time - start_time
    memory_taken = max(mem_usage) - min(mem_usage)
    
    return time_taken, memory_taken

# Path to the example image
img_path = '/Users/klz/Desktop/Cantay_Panorama/input_pano/example.jpg'
save_path = '/Users/klz/Desktop/Cantay_Panorama/output_pers/perspective.jpg'

# Parameters for GetPerspective
FOV = 120  # Wider field of view to capture more information
THETA = 0  # Adjust this to look left or right
PHI = 0    # Adjust this to look up or down
height = 2000  # Increase height to capture more detail
width = 3600   # Increase width to capture more detail
time_taken, memory_taken = profile_conversion(img_path, save_path, FOV, THETA, PHI, height, width)

print(f"Time taken: {time_taken:.2f} seconds")
print(f"Memory usage: {memory_taken:.2f} MiB")


Time taken: 2.09 seconds
Memory usage: 1137.72 MiB


In [ ]:
import os
import sys
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from time import time
from memory_profiler import memory_usage

def xyz2lonlat(xyz):
    atan2 = np.arctan2
    asin = np.arcsin

    norm = np.linalg.norm(xyz, axis=-1, keepdims=True)
    xyz_norm = xyz / norm
    x = xyz_norm[..., 0:1]
    y = xyz_norm[..., 1:2]
    z = xyz_norm[..., 2:]

    lon = atan2(x, z)
    lat = asin(y)
    lst = [lon, lat]

    out = np.concatenate(lst, axis=-1)
    return out

def lonlat2XY(lonlat, shape):
    X = (lonlat[..., 0:1] / (2 * np.pi) + 0.5) * (shape[1] - 1)
    Y = (lonlat[..., 1:] / (np.pi) + 0.5) * (shape[0] - 1)
    lst = [X, Y]
    out = np.concatenate(lst, axis=-1)

    return out 

class Equirectangular:
    def __init__(self, img_name):
        self._img = cv2.imread(img_name, cv2.IMREAD_COLOR)
        [self._height, self._width, _] = self._img.shape

    def GetPerspective(self, FOV, THETA, PHI):
        height = self._height
        width = self._width
        f = 0.5 * width / np.tan(0.5 * FOV / 180.0 * np.pi)
        cx = (width - 1) / 2.0
        cy = (height - 1) / 2.0
        K = np.array([
                [f, 0, cx],
                [0, f, cy],
                [0, 0,  1],
            ], np.float32)
        K_inv = np.linalg.inv(K)
        
        x = np.arange(width)
        y = np.arange(height)
        x, y = np.meshgrid(x, y)
        z = np.ones_like(x)
        xyz = np.concatenate([x[..., None], y[..., None], z[..., None]], axis=-1)
        xyz = xyz @ K_inv.T

        y_axis = np.array([0.0, 1.0, 0.0], np.float32)
        x_axis = np.array([1.0, 0.0, 0.0], np.float32)
        R1, _ = cv2.Rodrigues(y_axis * np.radians(THETA))
        R2, _ = cv2.Rodrigues(np.dot(R1, x_axis) * np.radians(PHI))
        R = R2 @ R1
        xyz = xyz @ R.T
        lonlat = xyz2lonlat(xyz) 
        XY = lonlat2XY(lonlat, shape=self._img.shape).astype(np.float32)
        persp = cv2.remap(self._img, XY[..., 0], XY[..., 1], cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)

        return persp

def evaluate_image_similarity(original_img, transformed_img):
    original_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    transformed_gray = cv2.cvtColor(transformed_img, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(original_gray, transformed_gray, full=True)
    return score

def find_optimal_params(img_path, save_path):
    best_params = None
    best_image = None
    best_quality = -1
    
    fov_range = range(60, 121, 10)  # FOV from 60 to 120 degrees
    theta_range = range(-90, 91, 30)  # THETA from -90 to 90 degrees
    phi_range = range(-45, 46, 15)  # PHI from -45 to 45 degrees
    
    original_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    height, width, _ = original_img.shape
    
    for FOV in fov_range:
        for THETA in theta_range:
            for PHI in phi_range:
                equi = Equirectangular(img_path)
                image = equi.GetPerspective(FOV, THETA, PHI)
                
                quality = evaluate_image_similarity(original_img, image)
                
                if quality > best_quality:
                    best_quality = quality
                    best_params = (FOV, THETA, PHI)
                    best_image = image
    
    if best_image is not None:
        cv2.imwrite(save_path, best_image)
    
    return best_params

# Path to the example image
img_path = '/Users/klz/Desktop/Cantay_Panorama/input_pano/example.jpg'
save_path = '/Users/klz/Desktop/Cantay_Panorama/output_pers/perspective.jpg'

best_params = find_optimal_params(img_path, save_path)

print(f"Optimal parameters: FOV={best_params[0]}, THETA={best_params[1]}, PHI={best_params[2]}")

